<a href="https://colab.research.google.com/github/crashidian/RandomScriptsForCS/blob/master/SchoolFacilitiesOptionsDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [49]:
class SchoolFacility:
    def __init__(self, name, condition, capacity, current_enrollment, age, annual_maintenance_cost):
        self.name = name
        self.condition = condition  # 0-100 scale
        self.capacity = capacity
        self.current_enrollment = current_enrollment
        self.age = age
        self.annual_maintenance_cost = annual_maintenance_cost

In [50]:
class SchoolDistrict:
    def __init__(self, name):
        self.name = name
        self.schools = []
        self.population_growth_rate = 0.01  # 1% annual growth
        self.bond_amount = 0

In [51]:
class Project:
    def __init__(self, name, school, capital_cost, annual_maintenance_impact, capacity_change, condition_improvement):
        self.name = name
        self.school = school
        self.capital_cost = capital_cost
        self.annual_maintenance_impact = annual_maintenance_impact  # Can be positive (cost increase) or negative (savings)
        self.capacity_change = capacity_change
        self.condition_improvement = condition_improvement

In [52]:
class Option:
    def __init__(self, name, projects):
        self.name = name
        self.projects = projects
        self.total_capital_cost = sum(project.capital_cost for project in projects)
        self.total_maintenance_impact = sum(project.annual_maintenance_impact for project in projects)
        self.total_capacity_change = sum(project.capacity_change for project in projects)
        self.average_condition_improvement = np.mean([project.condition_improvement for project in projects])

In [53]:
class MultiFacilityPlanningSystem:
    def __init__(self):
        self.district = None
        self.enrollment_model = None

    def load_district_data(self, district):
        self.district = district

    def predict_future_enrollment(self, years_ahead=5):
        X = pd.DataFrame([(school.age, school.capacity, school.condition) for school in self.district.schools],
                         columns=['age', 'capacity', 'condition'])
        y = pd.Series([school.current_enrollment for school in self.district.schools])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.enrollment_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.enrollment_model.fit(X_train, y_train)

        future_enrollments = []
        for school in self.district.schools:
            future_age = school.age + years_ahead
            future_condition = max(0, school.condition - (2 * years_ahead))
            future_enrollment = self.enrollment_model.predict([[future_age, school.capacity, future_condition]])[0]
            future_enrollment *= (1 + self.district.population_growth_rate) ** years_ahead
            future_enrollments.append(future_enrollment)

        return future_enrollments

    def generate_project_options(self):
        future_enrollments = self.predict_future_enrollment()

        options = []

        # Option 1: Minimal Intervention
        minimal_projects = []
        for school, future_enrollment in zip(self.district.schools, future_enrollments):
            if school.condition < 60:
                minimal_projects.append(Project(f"Essential repairs for {school.name}", school,
                                                capital_cost=2000000,
                                                annual_maintenance_impact=-50000,
                                                capacity_change=0,
                                                condition_improvement=20))
        options.append(Option("Minimal Intervention", minimal_projects))

        # Option 2: Moderate Renovation
        moderate_projects = []
        for school, future_enrollment in zip(self.district.schools, future_enrollments):
            if school.condition < 75:
                moderate_projects.append(Project(f"Moderate renovation of {school.name}", school,
                                                 capital_cost=10000000,
                                                 annual_maintenance_impact=-100000,
                                                 capacity_change=50,
                                                 condition_improvement=40))
        options.append(Option("Moderate Renovation", moderate_projects))

        # Option 3: Comprehensive Modernization
        comprehensive_projects = []
        for school, future_enrollment in zip(self.district.schools, future_enrollments):
            if future_enrollment > school.capacity or school.condition < 50:
                comprehensive_projects.append(Project(f"Comprehensive modernization of {school.name}", school,
                                                      capital_cost=25000000,
                                                      annual_maintenance_impact=-200000,
                                                      capacity_change=200,
                                                      condition_improvement=80))
        options.append(Option("Comprehensive Modernization", comprehensive_projects))

        # Option 4: New Construction
        if sum(future_enrollments) > sum(school.capacity for school in self.district.schools) * 1.1:
            new_school_project = Project("Construct new school", None,
                                         capital_cost=50000000,
                                         annual_maintenance_impact=500000,
                                         capacity_change=1000,
                                         condition_improvement=100)
            options.append(Option("New School Construction", [new_school_project]))

        return options

    def rank_options(self, options):
        def calculate_score(option):
            capital_cost_score = 1 / (option.total_capital_cost / 1e6)  # Lower cost is better
            maintenance_score = -option.total_maintenance_impact / 1e6  # Lower maintenance impact (or higher savings) is better
            capacity_score = option.total_capacity_change / 100  # Higher capacity increase is better
            condition_score = option.average_condition_improvement / 10  # Higher condition improvement is better

            # Weighted sum of scores
            return (capital_cost_score * 0.3 +
                    maintenance_score * 0.2 +
                    capacity_score * 0.3 +
                    condition_score * 0.2)

        ranked_options = sorted(options, key=calculate_score, reverse=True)
        return ranked_options

In [54]:
def main():
    # Create a sample school district
    district = SchoolDistrict("Sample Unified School District")
    district.schools = [
        SchoolFacility("High School A", 65, 1500, 1400, 30, 1000000),
        SchoolFacility("Middle School B", 55, 1000, 950, 40, 800000),
        SchoolFacility("Elementary School C", 80, 600, 550, 15, 500000),
        SchoolFacility("Elementary School D", 45, 500, 480, 55, 600000),
    ]
    district.bond_amount = 100000000  # $100 million bond

    # Initialize and run the multi-facility planning system
    mfps = MultiFacilityPlanningSystem()
    mfps.load_district_data(district)

    options = mfps.generate_project_options()
    ranked_options = mfps.rank_options(options)

    # Print results
    print(f"Multi-Option Facility Planning for {district.name}")
    print(f"Total Bond Amount Available: ${district.bond_amount:,}")
    print("\nRanked Options (from most viable to least viable):")

    for i, option in enumerate(ranked_options, 1):
        print(f"\n{i}. {option.name}")
        print(f"   Total Capital Cost: ${option.total_capital_cost:,}")
        print(f"   Annual Maintenance Impact: ${option.total_maintenance_impact:,}")
        print(f"   Total Capacity Change: {option.total_capacity_change}")
        print(f"   Average Condition Improvement: {option.average_condition_improvement:.2f}")
        print("   Projects:")
        for project in option.projects:
            print(f"   - {project.name}")
            print(f"     Capital Cost: ${project.capital_cost:,}")
            print(f"     Annual Maintenance Impact: ${project.annual_maintenance_impact:,}")
            print(f"     Capacity Change: {project.capacity_change}")
            print(f"     Condition Improvement: {project.condition_improvement}")

    # Provide a recommendation
    best_option = ranked_options[0]
    print(f"\nRecommendation:")
    print(f"Based on the analysis, the most viable option is: {best_option.name}")
    print(f"This option provides the best balance of capital costs, maintenance impact, capacity improvement, and overall condition enhancement.")
    if best_option.total_capital_cost > district.bond_amount:
        print(f"Note: This option exceeds the available bond amount by ${best_option.total_capital_cost - district.bond_amount:,}.")
        print("Consider phasing the projects or seeking additional funding sources.")
    else:
        print(f"This option is within the available bond amount, with ${district.bond_amount - best_option.total_capital_cost:,} remaining for contingencies or additional improvements.")

In [55]:
if __name__ == "__main__":
    main()

Multi-Option Facility Planning for Sample Unified School District
Total Bond Amount Available: $100,000,000

Ranked Options (from most viable to least viable):

1. Comprehensive Modernization
   Total Capital Cost: $50,000,000
   Annual Maintenance Impact: $-400,000
   Total Capacity Change: 400
   Average Condition Improvement: 80.00
   Projects:
   - Comprehensive modernization of Elementary School C
     Capital Cost: $25,000,000
     Annual Maintenance Impact: $-200,000
     Capacity Change: 200
     Condition Improvement: 80
   - Comprehensive modernization of Elementary School D
     Capital Cost: $25,000,000
     Annual Maintenance Impact: $-200,000
     Capacity Change: 200
     Condition Improvement: 80

2. Moderate Renovation
   Total Capital Cost: $30,000,000
   Annual Maintenance Impact: $-300,000
   Total Capacity Change: 150
   Average Condition Improvement: 40.00
   Projects:
   - Moderate renovation of High School A
     Capital Cost: $10,000,000
     Annual Maintenance

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
